In [3]:
import pip
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization,Softmax,InputLayer
import numpy as np
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile,Image

## Model Structure

In [4]:
classifier = Sequential()



classifier.add(Conv2D(filters = 8 ,strides=(1,1), kernel_size=(3,3),activation='relu',padding='same',input_shape=(32,32,1)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(filters = 16, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(filters = 32, kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())
classifier.add(Dense(120))
classifier.add(Dense(60,'Softmax'))


classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])



In [6]:
splitfolders.fixed("C:\\Users\Saunak\Desktop\Image_splitting_test_run\BanglaLekha-Isolated\Images", output="output",
    seed=1337, fixed=(1500, 437), oversample=False, group_prefix=None, move=False)

Copying files: 166105 files [20:54, 132.37 files/s]


In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 8)         80        
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 8)        32        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 16)        1168      
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 16)       64        
 hNormalization)                                                 
                                                        

In [6]:
train_folder = "output/train1/"
test_folder = "output/val"

In [10]:
ImageFile.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(shear_range = .2, rotation_range = 25)
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory(train_folder, target_size = (32, 32), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")
test_set = test_datagen.flow_from_directory(test_folder, target_size = (32, 32), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")


Found 90000 images belonging to 60 classes.
Found 26220 images belonging to 60 classes.


In [14]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                  patience=5, min_lr=0.0001)
classifier.fit(training_set ,steps_per_epoch = 90000, epochs = 10,
                validation_data = test_set, validation_steps = 20000,callbacks=[reduce_lr])

Epoch 1/10
90000/90000 [==============================] - 280s 3ms/step - loss: 0.4317 - accuracy: 0.8819 - val_loss: 0.6174 - val_accuracy: 0.8364 - lr: 2.0000e-04
Epoch 2/10
90000/90000 [==============================] - 281s 3ms/step - loss: 0.4281 - accuracy: 0.8817 - val_loss: 0.6374 - val_accuracy: 0.8350 - lr: 2.0000e-04
Epoch 3/10
90000/90000 [==============================] - 288s 3ms/step - loss: 0.4209 - accuracy: 0.8836 - val_loss: 0.7098 - val_accuracy: 0.8324 - lr: 2.0000e-04
Epoch 4/10
90000/90000 [==============================] - 277s 3ms/step - loss: 0.4199 - accuracy: 0.8840 - val_loss: 0.6340 - val_accuracy: 0.8288 - lr: 2.0000e-04
Epoch 5/10
90000/90000 [==============================] - 281s 3ms/step - loss: 0.4195 - accuracy: 0.8843 - val_loss: 0.6625 - val_accuracy: 0.8304 - lr: 2.0000e-04
Epoch 6/10
90000/90000 [==============================] - 280s 3ms/step - loss: 0.4162 - accuracy: 0.8845 - val_loss: 0.6510 - val_accuracy: 0.8275 - lr: 2.0000e-04
Epoch 7/10

KeyboardInterrupt: 

In [13]:
classifier_json = classifier.to_json()

with open("CNN_Bangla_Paper_Model.json", "w") as json_file:
    json_file.write(classifier_json)
    
classifier.save_weights("CNN_Bangla_Paper_Model.h5")
print('Saved model to disk')

Saved model to disk
